# Generador de ventas

Este script simula los datos de ventas de casa que, en una situación real, provendrían de un sitio web en el que un cliente marca como vendida una casa, o de programa que procesa un CSV de ventas cuando un vendedor sube el mismo al terminar la jornada.

In [ ]:
import pandas as pd
from confluent_kafka import Producer
import time
import json
import random

In [ ]:
from ejercicios.houses import SEED, TOPIC_SALES, TO_STREAM

In [ ]:
random.seed(SEED)

In [ ]:
ds = pd.read_csv(TO_STREAM)

In [ ]:
h = ds.sample(n=1)
d = h.to_dict('records')[0]

In [ ]:
h

Enviaremos los datos en JSON para facilitar la serialización/deserialización.

In [ ]:
d

In [ ]:
p = Producer({'bootstrap.servers': 'localhost:9092'})

def delivery_report(err, msg):
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

while True:
    # recuperamos los posibles callbacks
    p.poll(0)

    # escogemos una casa al azar y convertimos el registro en un diccionario
    # al enviarlo a Kafka lo serializamos como un string
    h = ds.sample(n=1)
    d = h.to_dict('records')[0]

    p.produce(TOPIC_SALES, json.dumps(d).encode('utf-8'), callback=delivery_report)

    #esperamos aprox. 2 segundos entre ventas
    time.sleep(abs(random.gauss(2, 1)))

p.flush()
